# Utilisation de l'augmentation des données pour améliorer les performances avec l'API Keras.

Plus souvent qu'autrement, nous pouvons bénéficier de fournir plus de données à notre modèle. Mais les données sont chères et rares. Existe-t-il un moyen de contourner cette limitation ? Oui il y a! Nous pouvons synthétiser de nouveaux exemples d'entraînement en effectuant de petites modifications sur ceux que nous avons déjà, tels que des rotations aléatoires, des recadrages aléatoires et des basculements horizontaux, entre autres. Dans cette recette, nous allons apprendre à utiliser l'augmentation de données avec l'API Keras pour améliorer les performances.

Dans cette recette, nous utiliserons le jeu de données Caltech 101, disponible ici : http://www.vision.caltech.edu/Image_Datasets/Caltech101/. Téléchargez et décompressez 101_ObjectCategories.tar.gz à votre emplacement préféré.

https://drive.google.com/u/0/uc?export=download&confirm=k34b&id=137RyRjvTBkBiIfeYBNZBtViDHQ6_Ewsp

In [6]:
#!wget -c http://www.vision.caltech.edu/Image_Datasets/Caltech101/101_ObjectCategories.tar.gz

In [7]:
#!tar xvf 101_ObjectCategories.tar.gz

**1.** Importez les modules requis :

In [8]:
#!pip install git+https://github.com/tensorflow/docs

In [2]:
import os
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
from glob import glob
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import *

**2.** Dénissez une fonction pour charger toutes les images de l'ensemble de données, ainsi que leurs étiquettes, en fonction de leurs chemins de fichiers :

In [3]:
def load_images_and_labels(image_paths, target_size=(64, 64)):
    images = []
    labels = []

    for image_path in image_paths:
        image = load_img(image_path, target_size=target_size)
        image = img_to_array(image)

        label = image_path.split(os.path.sep)[-2]

        images.append(image)
        labels.append(label)

    return np.array(images), np.array(labels)

**3.** Dénissez une fonction pour créer une version plus petite de VGG :

In [4]:
def build_network(width, height, depth, classes):
    input_layer = Input(shape=(width, height, depth))

    x = Conv2D(filters=32,
               kernel_size=(3, 3),
               padding='same')(input_layer)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Conv2D(filters=32,
               kernel_size=(3, 3),
               padding='same')(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(rate=0.25)(x)

    x = Conv2D(filters=64,
               kernel_size=(3, 3),
               padding='same')(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Conv2D(filters=64,
               kernel_size=(3, 3),
               padding='same')(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(rate=0.25)(x)

    x = Flatten()(x)
    x = Dense(units=512)(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Dropout(rate=0.25)(x)

    x = Dense(units=classes)(x)
    output = Softmax()(x)

    return Model(input_layer, output)

**4.** Déﬁnissez une fonction pour tracer et enregistrer la courbe d'entraînement d'un modèle :

In [9]:
def plot_model_history(model_history, metric, plot_name):
    plt.style.use('seaborn-darkgrid')
    plotter = tfdocs.plots.HistoryPlotter()
    plotter.plot({'Model': model_history}, metric=metric)

    plt.title(f'{metric.upper()}')
    plt.ylim([0, 1])

    plt.savefig(f'{plot_name}.png')
    plt.close()

**5.** Définissez la graine aléatoire :

In [10]:
SEED = 999
np.random.seed(SEED)

**6.** Chargez les chemins vers toutes les images de l'ensemble de données, à l'exception de celles de la classe BACKGROUND_Google :

In [11]:
base_path = (pathlib.Path('/content') / '101_ObjectCategories')
images_pattern = str(base_path / '*' / '*.jpg')
image_paths = [*glob(images_pattern)]
image_paths = [p for p in image_paths if
               p.split(os.path.sep)[-2] != 'BACKGROUND_Google']

**7.** Calculez l'ensemble des classes dans l'ensemble de données :

In [12]:
classes = {p.split(os.path.sep)[-2] for p in image_paths}

**8.** Chargez l'ensemble de données en mémoire, normalisez les images et encodez à one-hot les étiquettes :

In [13]:
X, y = load_images_and_labels(image_paths)
X = X.astype('float') / 255.0
y = LabelBinarizer().fit_transform(y)

**9.** Créez les sous-ensembles d'entraînement et de test :

In [14]:
(X_train, X_test,
 y_train, y_test) = train_test_split(X, y,
                                     test_size=0.2,
                                     random_state=SEED)

**10.** Construisez, compilez, entraînez et évaluez un réseau de neurones sans augmentation de données :

In [15]:
EPOCHS = 40
BATCH_SIZE = 64
model = build_network(64, 64, 3, len(classes))
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=EPOCHS,
                    batch_size=BATCH_SIZE)
result = model.evaluate(X_test, y_test)
print(f'Test accuracy: {result[1]}')

Epoch 1/40
109/109 [==============================] - 37s 66ms/step - loss: 2.6393 - accuracy: 0.4450 - val_loss: 5.1800 - val_accuracy: 0.0236
Epoch 2/40
109/109 [==============================] - 7s 60ms/step - loss: 1.2424 - accuracy: 0.6980 - val_loss: 5.4633 - val_accuracy: 0.0236
Epoch 3/40
109/109 [==============================] - 7s 60ms/step - loss: 0.5338 - accuracy: 0.8811 - val_loss: 2.9012 - val_accuracy: 0.3957
Epoch 4/40
109/109 [==============================] - 6s 59ms/step - loss: 0.2015 - accuracy: 0.9607 - val_loss: 2.0907 - val_accuracy: 0.5236
Epoch 5/40
109/109 [==============================] - 6s 59ms/step - loss: 0.1111 - accuracy: 0.9823 - val_loss: 1.6211 - val_accuracy: 0.6417
Epoch 6/40
109/109 [==============================] - 6s 60ms/step - loss: 0.0750 - accuracy: 0.9888 - val_loss: 1.8492 - val_accuracy: 0.6331
Epoch 7/40
109/109 [==============================] - 6s 60ms/step - loss: 0.0530 - accuracy: 0.9929 - val_loss: 1.6784 - val_accuracy: 0.646

**11.** Construisez, compilez, entraînez et évaluez le même réseau, cette fois avec l'augmentation des données :

In [18]:
model = build_network(64, 64, 3, len(classes))
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

augmenter = ImageDataGenerator(horizontal_flip=True,
                               rotation_range=30,
                               width_shift_range=0.1,
                               height_shift_range=0.1,
                               shear_range=0.2,
                               zoom_range=0.2,
                               fill_mode='nearest')
train_generator = augmenter.flow(X_train, y_train, BATCH_SIZE)
history = model.fit(train_generator,
                    steps_per_epoch=len(X_train) // BATCH_SIZE,
                    validation_data=(X_test, y_test),
                    epochs=EPOCHS)
result = model.evaluate(X_test, y_test)
print(f'Test accuracy: {result[1]}')
plot_model_history(history, 'accuracy', 'augmented')

Epoch 1/40
108/108 [==============================] - 14s 111ms/step - loss: 3.3722 - accuracy: 0.3157 - val_loss: 5.1492 - val_accuracy: 0.1008
Epoch 2/40
108/108 [==============================] - 12s 113ms/step - loss: 2.6713 - accuracy: 0.4105 - val_loss: 4.4853 - val_accuracy: 0.1394
Epoch 3/40
108/108 [==============================] - 12s 109ms/step - loss: 2.3089 - accuracy: 0.4687 - val_loss: 3.5878 - val_accuracy: 0.2373
Epoch 4/40
108/108 [==============================] - 12s 114ms/step - loss: 2.0770 - accuracy: 0.5104 - val_loss: 2.3261 - val_accuracy: 0.4758
Epoch 5/40
108/108 [==============================] - 12s 113ms/step - loss: 1.9038 - accuracy: 0.5389 - val_loss: 1.9020 - val_accuracy: 0.5622
Epoch 6/40
108/108 [==============================] - 12s 113ms/step - loss: 1.8023 - accuracy: 0.5610 - val_loss: 4.0087 - val_accuracy: 0.4562
Epoch 7/40
108/108 [==============================] - 12s 111ms/step - loss: 1.7287 - accuracy: 0.5764 - val_loss: 1.9902 - val_ac

En comparant les étapes 10 et 11, nous observons un gain notable de performances en utilisant l'augmentation de données. Comprenons mieux ce que nous avons fait dans la section suivante

Dans cette recette, nous avons implémenté une version réduite de VGG sur le jeu de données Caltech 101 difficile. Tout d'abord, nous avons formé un réseau uniquement sur les données d'origine, puis en utilisant l'augmentation des données. Le premier réseau (voir étape 10) a obtenu un niveau de précision sur le jeu de test de 62,3 % et montre clairement des signes de dépassement, car l'écart qui sépare les courbes de précision d'apprentissage et de validation est très large. D'autre part, en appliquant une série de perturbations aléatoires, via ImageDataGenerator(), telles que des basculements horizontaux, des rotations, des changements de largeur et de hauteur, entre autres (voir l'étape 11), nous avons augmenté la précision sur l'ensemble de test à 66,4 %. De plus, l'écart entre les courbes de précision d'apprentissage et de validation est beaucoup plus faible cette fois, ce qui suggère un effet de régularisation résultant de l'application de l'augmentation des données.